# การสร้างข้อมูล Preference ด้วย ThaiLLM API

## สารบัญ
1. [บทนำ](#1-บทนำ)
2. [ขอ API Key จาก ThaiLLM](#2-ขอ-api-key-จาก-thaillm)
3. [เชื่อมต่อ ThaiLLM API](#3-เชื่อมต่อ-thaillm-api)
4. [สร้างข้อมูล Preference](#4-สร้างข้อมูล-preference)
5. [บันทึกและใช้งานข้อมูล](#5-บันทึกและใช้งานข้อมูล)

---
## 1. บทนำ

ใน notebook นี้ เราจะใช้ **ThaiLLM API** (OpenThaiGPT) ในการสร้างข้อมูล Preference สำหรับ DPO

### ทำไมต้องใช้ ThaiLLM?

- **OpenThaiGPT** เป็น LLM ที่พัฒนาสำหรับภาษาไทยโดยเฉพาะ
- ใช้งานผ่าน API ได้ง่าย ไม่ต้องติดตั้งโมเดลบนเครื่อง
- เหมาะสำหรับการสร้างข้อมูลภาษาไทยคุณภาพสูง

### รูปแบบข้อมูล Preference

```json
{
    "instruction": "คำสั่งหรือคำถาม",
    "input": "ข้อมูลเพิ่มเติม (optional)",
    "chosen": "คำตอบที่ดี (สุภาพ ละเอียด เป็นประโยชน์)",
    "rejected": "คำตอบที่ไม่ดี (สั้น ไม่สุภาพ)"
}
```

---
## 2. ขอ API Key จาก ThaiLLM

### ขั้นตอนการขอ API Key:

1. ไปที่ **https://playground.thaillm.or.th**
2. ลงทะเบียนหรือเข้าสู่ระบบ
3. ขอ API Key จากระบบ
4. เก็บ API Key ไว้ในที่ปลอดภัย

⚠️ **สำคัญ:** อย่าเปิดเผย API Key ในโค้ดที่แชร์สาธารณะ!

In [ ]:
# ตั้งค่า API Key
# วิธีที่ 1: ใส่โดยตรง (ไม่แนะนำสำหรับโค้ดที่แชร์)
# THAILLM_API_KEY = "your-api-key-here"

# วิธีที่ 2: ใช้ environment variable (แนะนำ)
import os
THAILLM_API_KEY = os.environ.get("THAILLM_API_KEY", "")

# วิธีที่ 3: ใช้ getpass (ปลอดภัยสำหรับ notebook)
if not THAILLM_API_KEY:
    from getpass import getpass
    THAILLM_API_KEY = getpass("กรุณาใส่ ThaiLLM API Key: ")

print("✅ API Key ถูกตั้งค่าแล้ว" if THAILLM_API_KEY else "❌ ยังไม่ได้ตั้งค่า API Key")

---
## 3. เชื่อมต่อ ThaiLLM API

ThaiLLM API ใช้รูปแบบคล้าย OpenAI API

In [ ]:
import requests
import json

# ThaiLLM API endpoint
THAILLM_API_URL = "http://thaillm.or.th/api/openthaigpt/v1/chat/completions"

def query_thaillm(prompt, max_tokens=2048, temperature=0.7):
    """
    ส่ง prompt ไปหา ThaiLLM API และรับคำตอบกลับมา
    
    Parameters:
    -----------
    prompt: str
        ข้อความที่จะส่งไป
    max_tokens: int
        จำนวน tokens สูงสุดในคำตอบ (default: 2048)
    temperature: float
        ความสุ่มของคำตอบ 0-1 (default: 0.7)
    
    Returns:
    --------
    str: คำตอบจากโมเดล หรือ None ถ้าเกิดข้อผิดพลาด
    """
    headers = {
        "Content-Type": "application/json",
        "apikey": THAILLM_API_KEY
    }
    
    data = {
        "model": "/model",
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "max_tokens": max_tokens,
        "temperature": temperature
    }
    
    try:
        response = requests.post(
            THAILLM_API_URL,
            headers=headers,
            json=data,
            timeout=60
        )
        response.raise_for_status()
        
        result = response.json()
        return result["choices"][0]["message"]["content"]
    
    except requests.exceptions.Timeout:
        print("❌ Error: Request timeout")
        return None
    except requests.exceptions.RequestException as e:
        print(f"❌ Error: {e}")
        return None
    except (KeyError, IndexError) as e:
        print(f"❌ Error parsing response: {e}")
        return None

In [ ]:
# ทดสอบการเชื่อมต่อ
print("กำลังทดสอบการเชื่อมต่อ ThaiLLM API...")
print("-" * 50)

test_response = query_thaillm("สวัสดีครับ แนะนำตัวหน่อย")

if test_response:
    print("✅ เชื่อมต่อ ThaiLLM API สำเร็จ!")
    print(f"\n📝 คำตอบ:\n{test_response}")
else:
    print("❌ ไม่สามารถเชื่อมต่อ ThaiLLM API ได้")
    print("\nกรุณาตรวจสอบ:")
    print("1. API Key ถูกต้อง")
    print("2. มีการเชื่อมต่ออินเทอร์เน็ต")
    print("3. ขอ API Key ได้ที่: https://playground.thaillm.or.th")

---
## 4. สร้างข้อมูล Preference

เราจะใช้ ThaiLLM สร้างคู่ของ chosen/rejected responses

### กลยุทธ์:
- **Chosen**: ให้ LLM ตอบแบบสุภาพ ละเอียด เป็นประโยชน์
- **Rejected**: ให้ LLM ตอบแบบสั้น ไม่สุภาพ หรือไม่ครบถ้วน

In [ ]:
def format_input(entry):
    """
    จัดรูปแบบ prompt สำหรับ instruction format
    """
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )
    
    input_text = f"\n\n### Input:\n{entry['input']}" if entry.get("input") else ""
    
    return instruction_text + input_text

In [ ]:
import random

def generate_preference_pair_thaillm(entry):
    """
    สร้างคู่ chosen/rejected โดยใช้ ThaiLLM API
    
    Parameters:
    -----------
    entry: dict
        ข้อมูลที่มี instruction และ input
    
    Returns:
    --------
    dict: entry ที่มี chosen และ rejected
    """
    instruction = entry.get("instruction", "")
    input_text = entry.get("input", "")
    
    # สร้าง prompt สำหรับ chosen (คำตอบที่ดี)
    chosen_prompt = f"""คุณเป็นผู้ช่วยที่สุภาพและเป็นประโยชน์ กรุณาตอบคำถามต่อไปนี้อย่างละเอียดและเป็นมิตร:

คำถาม: {instruction}
{f'ข้อมูลเพิ่มเติม: {input_text}' if input_text else ''}

กรุณาตอบเป็นภาษาไทย อย่างสุภาพและครบถ้วน:"""
    
    # สร้าง prompt สำหรับ rejected (คำตอบที่ไม่ดี)
    rejected_prompt = f"""ตอบคำถามต่อไปนี้แบบสั้นมากๆ ไม่ต้องอธิบาย ตอบแค่ 1-2 ประโยค:

คำถาม: {instruction}
{f'ข้อมูลเพิ่มเติม: {input_text}' if input_text else ''}

ตอบสั้นๆ:"""
    
    # เรียก API สำหรับ chosen
    chosen_response = query_thaillm(chosen_prompt, temperature=0.7)
    
    # เรียก API สำหรับ rejected  
    rejected_response = query_thaillm(rejected_prompt, temperature=0.3)
    
    # สร้าง entry ใหม่
    result = {
        "instruction": instruction,
        "input": input_text,
        "chosen": chosen_response if chosen_response else "ขออภัย ไม่สามารถสร้างคำตอบได้",
        "rejected": rejected_response if rejected_response else instruction[:50]
    }
    
    return result

In [ ]:
# ทดสอบสร้าง preference pair
test_entry = {
    "instruction": "อธิบายว่า Machine Learning คืออะไร",
    "input": ""
}

print("กำลังสร้าง Preference Pair...")
print("="*60)

result = generate_preference_pair_thaillm(test_entry)

print(f"\n📝 Instruction: {result['instruction']}")
print(f"\n✅ Chosen (คำตอบที่ดี):")
print(result['chosen'])
print(f"\n❌ Rejected (คำตอบที่ไม่ดี):")
print(result['rejected'])

In [ ]:
from tqdm import tqdm
import time

def generate_preference_dataset_thaillm(instructions, delay=1.0):
    """
    สร้าง preference dataset จาก list ของ instructions
    
    Parameters:
    -----------
    instructions: list
        รายการของ instruction entries
    delay: float
        เวลาหน่วง (วินาที) ระหว่างแต่ละ request เพื่อไม่ให้เกิน rate limit
    
    Returns:
    --------
    list: preference dataset
    """
    preference_data = []
    
    for entry in tqdm(instructions, desc="กำลังสร้างข้อมูล"):
        result = generate_preference_pair_thaillm(entry)
        preference_data.append(result)
        
        # หน่วงเวลาเพื่อไม่ให้เกิน rate limit
        time.sleep(delay)
    
    return preference_data

In [ ]:
# ตัวอย่าง instructions ภาษาไทย
sample_instructions = [
    {"instruction": "อธิบายเกี่ยวกับประเทศไทยสั้นๆ", "input": ""},
    {"instruction": "วิธีทำต้มยำกุ้ง", "input": ""},
    {"instruction": "แปลประโยคต่อไปนี้เป็นภาษาอังกฤษ", "input": "ฉันรักประเทศไทย"},
    {"instruction": "อธิบายหลักการทำงานของ Neural Network", "input": ""},
    {"instruction": "แนะนำวิธีเรียนภาษาอังกฤษ", "input": ""},
]

print(f"จำนวน instructions: {len(sample_instructions)}")

In [ ]:
# สร้าง preference dataset
# หมายเหตุ: อาจใช้เวลาสักครู่ขึ้นอยู่กับจำนวนข้อมูล

print("เริ่มสร้าง Preference Dataset...")
print("="*60)

preference_dataset = generate_preference_dataset_thaillm(
    sample_instructions,
    delay=1.0  # หน่วง 1 วินาทีระหว่างแต่ละ request
)

print(f"\n✅ สร้างข้อมูลเสร็จ: {len(preference_dataset)} รายการ")

In [ ]:
# แสดงตัวอย่างผลลัพธ์
for i, entry in enumerate(preference_dataset[:3]):
    print("="*60)
    print(f"ตัวอย่างที่ {i+1}")
    print("="*60)
    print(f"\n📝 Instruction: {entry['instruction']}")
    if entry['input']:
        print(f"📥 Input: {entry['input']}")
    print(f"\n✅ Chosen:\n{entry['chosen'][:300]}..." if len(entry['chosen']) > 300 else f"\n✅ Chosen:\n{entry['chosen']}")
    print(f"\n❌ Rejected:\n{entry['rejected']}")
    print()

---
## 5. บันทึกและใช้งานข้อมูล

In [ ]:
def save_preference_data(data, filename):
    """
    บันทึกข้อมูลเป็นไฟล์ JSON
    """
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"✅ บันทึกข้อมูลไปที่: {filename}")
    print(f"   จำนวนข้อมูล: {len(data)} รายการ")

In [ ]:
# บันทึกข้อมูล
save_preference_data(preference_dataset, "my_thaillm_preference_data.json")

In [ ]:
# โหลดข้อมูลกลับมา
def load_preference_data(filename):
    """
    โหลดข้อมูลจากไฟล์ JSON
    """
    with open(filename, "r", encoding="utf-8") as f:
        data = json.load(f)
    print(f"✅ โหลดข้อมูลจาก: {filename}")
    print(f"   จำนวนข้อมูล: {len(data)} รายการ")
    return data

# loaded_data = load_preference_data("my_thaillm_preference_data.json")

---
## สรุป

### สิ่งที่เราได้เรียนรู้:

1. **ThaiLLM API**: วิธีเชื่อมต่อและใช้งาน OpenThaiGPT ผ่าน API

2. **การสร้าง Preference Data**: 
   - ใช้ prompt ต่างกันสำหรับ chosen และ rejected
   - Chosen: สุภาพ ละเอียด เป็นประโยชน์
   - Rejected: สั้น ไม่ครบถ้วน

3. **Best Practices**:
   - เก็บ API Key ปลอดภัย (ใช้ environment variable หรือ getpass)
   - ใส่ delay ระหว่าง requests เพื่อไม่ให้เกิน rate limit
   - ตรวจสอบคุณภาพข้อมูลก่อนใช้งาน

### ข้อควรระวัง:

- API อาจมี rate limit
- ควรตรวจสอบคุณภาพข้อมูลที่สร้างได้
- อย่าเปิดเผย API Key

---

**ขั้นตอนถัดไป**: ไปยัง `01_dpo_from_scratch_thai.ipynb` เพื่อฝึกโมเดลด้วย DPO

---

**แหล่งข้อมูล:**

- ThaiLLM Playground: https://playground.thaillm.or.th
- Raschka, Sebastian. *Build A Large Language Model (From Scratch)*. Manning, 2024.
- https://github.com/rasbt/LLMs-from-scratch